In [ ]:
lnx=[a.strip() for a in open("/home/limu/Data/Quvik/targets_intermediate_mass_mergers.txt").readlines()]      
lnx=[a for a in lnx if len(a)>10 and a[0]!='#'] 
set1=[a.split('\t') for a in lnx if a.find('\t')>0][1:] 

pos=[coord.ICRS(Angle(a[-1][:mpos(a[-1])].strip()+'h'),Angle(a[-1][mpos(a[-1]):40].strip()+'d')) for a in set1]
names=[a[1] if a[0][:4]=='IRAS' and a[1].strip()!='' else a[0] for a in set1] 
targ=[md.Target(name=t.strip(),group='medmerg:',mag1=0,mag2=0,mag3=0,mag0=0) for t in names]
[targ[i].set_pos(pos[i]) for i in range(len(pos))]

oo=[md.Objective(expo=10,repeat=3*24*3600/10,target=t) for t in targ[4:6]]

#all have objectives
[t for t in md.Target.objects.all() if t.objective_set.count()==0]

In [4]:
import json,requests
hscapiurl = "https://catalogs.mast.stsci.edu/api/v0.1/hsc"

def cat2url(table="hcvsummary",release="v3",magtype="magaper2",baseurl=hscapiurl):
    #checklegal(table,release,magtype)
    if table == "summary":
        url = "{baseurl}/{release}/{table}/{magtype}".format(**locals())
    else:
        url = "{baseurl}/{release}/{table}".format(**locals())
    return url

def get_hla_cutout(imagename,ra,dec,size=33,autoscale=99.5,asinh=True,zoom=1):
    """Get JPEG cutout for an image"""
    if imagename==None:
        from astropy.io import ascii
        tab=ascii.read(hcvcone(ra,dec,radius,table="hcvsummary"))

    url = "https://hla.stsci.edu/cgi-bin/fitscut.cgi"
    r = requests.get(url, params=dict(ra=ra, dec=dec, size=size, 
            format="jpeg", red=imagename, autoscale=autoscale, asinh=asinh, zoom=zoom))
    im = Image.open(BytesIO(r.content))
    return im


def mastQuery(request, url='https://mast.stsci.edu/api/v0/invoke'):
    requestString = json.dumps(request)
    r = requests.post(url, data={'request': requestString})
    r.raise_for_status()
    return r.text

def resolve(name):
    resolverRequest = {'service':'Mast.Name.Lookup',
                       'params':{'input':name,'format':'json'},
                      }
    resolvedObjectString = mastQuery(resolverRequest)
    resolvedObject = json.loads(resolvedObjectString)
    try:
        objRa = resolvedObject['resolvedCoordinate'][0]['ra']
        objDec = resolvedObject['resolvedCoordinate'][0]['decl']
    except IndexError as e:
        raise ValueError("Unknown object '{}'".format(name))
    return (objRa, objDec)

In [22]:
#resolve('EPIC 5481390')
from astroquery.mast import Observations
obs_table = Observations.query_object("EPIC 5481390",radius=".2 arcsec")

ResolverError: Could not resolve EPIC 5481390 to a sky position.

In [14]:
table2=Observations.query_criteria(filters=["*UV","GALEX"],objectname="M101")

In [18]:
len(obs_table)

266

In [21]:

import numpy as np
np.array(table2['distance'])

array([  0.        ,   0.        , 371.71837196, 371.71837196,
         0.        ,   0.        ,   3.83290685,   3.83290685,
         0.        ,   0.        , 229.81061601, 229.81061601])